# Modelling and Analysis of a Simple Unbalanced LV Network with a Single Wire Earth Return System (SWER)

## Introduction

This tutorial will demonstrate how to model an unbalanced LV network with a single wire earth return system in 
_Roseau Load Flow (RLF)_ solver. We will replicate the network which was initially specified in _OpenDSS_ syntax in 
_RLF_ and compare the results obtained to benchmark their capabilities.

Before attempting this tutorial, you should have finished Tutorial 1 in this repository for a basic knowledge of 
how the _RLF_ solver works. We'll be using a modified form of the network in Tutorial 1 that uses an earth return 
system instead of a neutral conductor as shown in Figure 1 below. The network consists of an MV bus, a MV/LV, 
$\Delta$-Y transformer (11kV/0.4kV, 250 kVA) between the source bus and bus A, a 240 mm² 3-phase line connecting 
buses A and B, and three 16 mm² single-phase lines connecting bus B with buses C, D and E each of which serves as 
a connection point for a house.

<center> <img style="float: middle;" 
          src="../images/LV_Network_Without_Neutral.png" 
          alt="Simple LV network"
          width="40%"> 
</center>

**<center> Figure 1. Simple LV Network with Earth Return System </center>**

The details for the loads in the network are given in the table below.
| Load Name | Phases | Connected bus | Peak Demand (kW) | PF |
| :-------- | :----- | :------------ | :--------------- | :--- |
| Load_1 | 1 | C | 7 | 0.95 |
| Load_2 | 1 | D | 6 | 0.95 |
| Load_3 | 1 | E | 8 | 0.95 |


## Import Required Packages


In [ ]:
import dss
import numpy as np
import roseau.load_flow as rlf
from rich.table import Table
from rich.text import Text

## Open DSS

Because this tutorial focuses on the _RLF_ solver, we will only provide a brief overview of how to model the above 
network in _OpenDSS_. For a detailed explanation of how to model this network in _OpenDSS_, we refer you to this origin 
of this tutorial which is available [here](https://github.com/Team-Nando/Tutorial-DERHostingCapacity-1-AdvancedTools_LV).


### Modelling the Network


In [ ]:
# Set up dss_engine
dss_engine = dss.DSS
DSSText = dss_engine.Text
DSSCircuit = dss_engine.ActiveCircuit
DSSSolution = dss_engine.ActiveCircuit.Solution
ControlQueue = dss_engine.ActiveCircuit.CtrlQueue
dss_engine.AllowForms = 0

In [ ]:
# Network Modelling - Creating a Circuit
DSSText.Command = "Clear"
DSSText.Command = "Set DefaultBaseFrequency=50"
DSSText.Command = "New Circuit.Simple_LV_Network"
DSSText.Command = "Edit vsource.source bus1=sourceBus basekv=11 pu=1.0 phases=3"

In [ ]:
# Adding the 11kV/0.4kV Transformer
DSSText.Command = "New transformer.LVTR Buses=[sourcebus, A.1.2.3] Conns=[delta wye] KVs=[11, 0.4] KVAs=[250 250] %Rs=0.00 xhl=2.5 %loadloss=0 "
# DSSText.Command = "dump transformer.LVTR debug"

In [ ]:
# Creating the linecodes
DSSText.Command = "new linecode.240sq nphases=3 R1=0.127 X1=0.072 R0=0.342 X0=0.089 units=km"
DSSText.Command = "new linecode.16sq nphases=1 R1=1.15 X1=0.083 R0=1.2 X0=0.083 units=km"

# Creating the 400V and 230V lines
DSSText.Command = "new line.A_B bus1=A.1.2.3 bus2=B.1.2.3 length=1 phases=3 units=km linecode=240sq"
DSSText.Command = "new line.B_L1 bus1=B.1 bus2=C.1 length=0.01 phases=1 units=km linecode=16sq"
DSSText.Command = "new line.B_L2 bus1=B.2 bus2=D.1 length=0.01 phases=1 units=km linecode=16sq"
DSSText.Command = "new line.B_L3 bus1=B.3 bus2=E.1 length=0.01 phases=1 units=km linecode=16sq"

In [ ]:
# Connecting loads to a bus
DSSText.Command = "new load.Load_1 bus1=C.1 phases=1 kV=(0.4 3 sqrt /) kW=7 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_2 bus1=D.1 phases=1 kV=(0.4 3 sqrt /) kW=6 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"
DSSText.Command = "new load.Load_3 bus1=E.1 phases=1 kV=(0.4 3 sqrt /) kW=8 pf=0.95 model=1 conn=wye Vminpu=0.85 Vmaxpu=1.20 status=fixed"

In [ ]:
# Set the Control mode and the Voltage bases
DSSText.Command = "set controlmode=static"
DSSText.Command = "set mode=snapshot"
DSSText.Command = "Set VoltageBases=[11 0.4]"
DSSText.Command = "calcvoltagebases"

### Running a Load Flow Simulation


In [ ]:
# Run the power flow simulation
DSSSolution.Solve()
if DSSSolution.Converged:
    print("The Circuit was Successfully Solved")
else:
    raise RuntimeError("DID NOT CONVERGE")

### Accessing Results


In [ ]:
# Extract active and reactive power of loads
table_titles = []
table_opendss_results = []
active_power_demand_title = "Active power demand (P)"
reactive_power_demand_title = "Reactive power demand (Q)"
for active_load in ("Load_1", "Load_2", "Load_3"):
    table_titles.append(f"Power - {active_load}")
    table_opendss_results.append(None)

    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    print(f"{active_load}:  ")
    pp, pn = DSSCircuit.ActiveElement.Powers[0::2]
    active_power_demand = f"{pp+pn:.3f} kW"
    print(f"{active_power_demand_title} = {active_power_demand} ")
    table_titles.append(active_power_demand_title)
    table_opendss_results.append(active_power_demand)

    qp, qn = DSSCircuit.ActiveElement.Powers[1::2]
    np.testing.assert_allclose([pn, qn], 0)  # For some reason they have 2 values for P&Q !!
    reactive_power_demand = f"{qp+qn:.3f} kVAr"
    print(f"{reactive_power_demand_title} = {reactive_power_demand}")
    table_titles.append(reactive_power_demand_title)
    table_opendss_results.append(reactive_power_demand)

In [ ]:
# Extract load bus voltages
for active_load in ("Load_1", "Load_2", "Load_3"):
    table_titles.append(f"Voltage - {active_load}")
    table_opendss_results.append(None)

    DSSCircuit.SetActiveElement(f"Load.{active_load}")
    bus_name = DSSCircuit.ActiveElement.Properties("bus1").Val
    DSSCircuit.SetActiveBus(bus_name)
    voltages = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
    (v1,) = np.abs(voltages)
    voltage_title = f"The voltage of the bus connected to {active_load}"
    voltage_value = f"{v1:.3f} pu"
    print(f"{voltage_title} = {voltage_value}")
    table_titles.append(voltage_title)
    table_opendss_results.append(voltage_value)

In [ ]:
# Extract bus A voltages
active_bus = "A"
DSSCircuit.SetActiveBus(active_bus)
print(f"Voltage magnitudes at bus {active_bus}:  ")
table_titles.append(f"Bus {active_bus}")
table_opendss_results.append(None)
voltages = DSSCircuit.ActiveBus.puVoltages[0::2] + 1j * DSSCircuit.ActiveBus.puVoltages[1::2]
v1, v2, v3 = np.abs(voltages)
for i, v in enumerate((v1, v2, v3)):
    voltage_title = f"Voltage magnitude - phase {i + 1}"
    table_titles.append(voltage_title)
    voltage_value = f"{v:.3f} pu"
    table_opendss_results.append(voltage_value)
    print(f"{voltage_title} = {voltage_value}")

In [ ]:
# Extract transformer active and reactive power as well as losses
DSSCircuit.SetActiveElement("transformer.LVTR")
print("Results of the transformer LVTR: ")
table_titles.append("Results of the transformer LVTR:")
table_opendss_results.append(None)
# Indices [0,4] are primary powers, indices [4, 8] are secondary powers
transformer_p = DSSCircuit.ActiveElement.Powers[0::2]
transformer_q = DSSCircuit.ActiveElement.Powers[1::2]


for i in range(3):
    transformer_title = f"Active power (P) supplied to phase {i+1}"
    transformer_value = f"{abs(transformer_p[4+i]):.5f} kW"
    table_titles.append(transformer_title)
    table_opendss_results.append(transformer_value)
    print(f"{transformer_title} = {transformer_value}")

for i in range(3):
    transformer_title = f"Reactive power (Q) supplied to phase {i+1}"
    transformer_value = f"{abs(transformer_q[4+i]):.5f} kVAr"
    table_titles.append(transformer_title)
    table_opendss_results.append(transformer_value)
    print(f"{transformer_title} = {transformer_value}")

In [ ]:
# Extract line losses
line_loss = DSSCircuit.LineLosses
lines_p, lines_q = line_loss
print("Results of the Power Losses: ")
table_titles.append("Line Losses")
table_opendss_results.append(None)

line_title = "Active power (P) losses"
line_value = f"{abs(lines_p):.3f} kW"
table_titles.append(line_title)
table_opendss_results.append(line_value)
print(f"{line_title} = {line_value}")

line_title = "Reactive power (Q) losses"
line_value = f"{abs(lines_q):.3f} kVAr"
table_titles.append(line_title)
table_opendss_results.append(line_value)
print(f"{line_title} = {line_value}")

## Roseau Load Flow

We will now model the same network using the _RLF_ package. As demonstrated in tutorial 1, the workflow for network 
modelling in _RLF_ typically starts with modelling the buses which we do as shown below using the 
[`Bus`](https://roseau-load-flow.roseautechnologies.com/models/Bus.html) class.


In [ ]:
# Buses
source_bus = rlf.Bus(id="source_bus", phases="abc")
bus_a = rlf.Bus(id="bus_A", phases="abcn")
bus_b = rlf.Bus(id="bus_B", phases="abcn")
bus_c = rlf.Bus(id="bus_C", phases="an")
bus_d = rlf.Bus(id="bus_D", phases="bn")
bus_e = rlf.Bus(id="bus_E", phases="cn")

Next, we create the voltage source attached to the source bus using the 
[`VoltageSource`](https://roseau-load-flow.roseautechnologies.com/models/VoltageSource.html) class and specifying its 
voltage values. Here we are using 11 kV line-line since the source bus is on the MV side of the network.


In [ ]:
# Voltage Source
un_mv = rlf.Q_(11, "kV")
un_lv = rlf.Q_(400, "V")
vs = rlf.VoltageSource(id="vs", bus=source_bus, voltages=un_mv)
# `voltages=un_mv` is equivalent to `voltages=un_mv * np.exp([0, -2j * np.pi / 3, 2j * np.pi / 3])`

In the third step, we instantiate the transformer. However, since transformers in _RLF_ and _OpenDSS_ are specified 
differently, we will make use of the method `from_open_dss` of the 
[`TransformerParameters`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) class to 
ensure we replicate the same transformer. This method converts _OpenDSS_ transformer parameters into _RLF_ parameters. 
Once the conversion is done, we can use these parameters to create a 
[`Transformer`](https://roseau-load-flow.roseautechnologies.com/models/Transformer/index.html) object as shown below.


In [ ]:
# Converting Transformer Parameters
tp = rlf.TransformerParameters.from_open_dss(
    id="LVTR",
    conns=("delta", "wye"),
    kvs=[un_mv, un_lv],
    kvas=(250, 250),
    leadlag="euro",  # <- should be "ansi" (i.e. "Dyn1") but we don't have this in RLF yet
    xhl=2.5,
    rs=0,
    loadloss=0,
    noloadloss=0,
    imag=0,
    normhkva=None,
)

# Transformer
transformer = rlf.Transformer("LVTR", bus1=source_bus, bus2=bus_a, parameters=tp)

Next, we will add potential references 
([`PotentialRef`](https://roseau-load-flow.roseautechnologies.com/models/PotentialRef.html)) which are unique elements 
required by the _RLF_ solver and are added to each galvanically isolated section of the network. For the MV side, we 
can simply add a potential reference to the source bus, and we don't need to worry about a neutral or the ground 
since the MV side is typically balanced.

However, for the LV side which does not have a neutral and can be unbalanced, we will need to create 
a [`Ground`](https://roseau-load-flow.roseautechnologies.com/models/Ground.html) element. This ground will be 
connected to the neutral of all the LV buses and will serve as the return path in the place of an actual neutral 
conductor. Finally, we will connect the LV 
([`PotentialRef`](https://roseau-load-flow.roseautechnologies.com/models/PotentialRef.html)) to this ground which will 
set its voltage to 0 V.


In [ ]:
# MV Potential Reference
pref_mv = rlf.PotentialRef(id="pref_mv", element=source_bus)

# LV Ground and Potential Reference
ground = rlf.Ground(id="ground")
# NOTE THE GROUND CONNECTION TO ALL BUSES, that is because our lines don't have neutrals
# and we want the current to return through the earth
for b in (bus_a, bus_b, bus_c, bus_d, bus_e):
    ground.connect(bus=b, phase="n")
pref_lv = rlf.PotentialRef(id="pref_lv", element=ground)

In the fifth step, we will create the lines in the network. Similar to the transformers, we would like to replicate the 
same lines from the OpenDSS specifications in _RLF_ to ensure consistency. To do this, we will employ the 
`from_open_dss` method of the 
[`LineParameters`](https://roseau-load-flow.roseautechnologies.com/models/Line/Parameters.html) class. This method 
converts the parameters of an _OpenDSS_ line (linecodes) into an _RLF_ 
[`LineParameters`](https://roseau-load-flow.roseautechnologies.com/models/Line/Parameters.html) object. We do this for
 both types of lines in the network.

Then, we can use these converted 
[`LineParameters`](https://roseau-load-flow.roseautechnologies.com/models/Line/Parameters.html) to create the lines 
using the [`Line`](https://roseau-load-flow.roseautechnologies.com/models/Line/index.html) class as shown below. 
Compared to tutorial 1, note that we specify the phase names without an "n" to signify the lack of a neutral conductor.


In [ ]:
# Converting the OpenDSS linecodes into RLF line parameters
lp_240 = rlf.LineParameters.from_open_dss(
    id="linecode-240sq",
    nphases=3,
    r1=rlf.Q_(0.127, "ohm/km"),
    x1=rlf.Q_(0.072, "ohm/km"),
    r0=rlf.Q_(0.342, "ohm/km"),
    x0=rlf.Q_(0.089, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)
lp_16 = rlf.LineParameters.from_open_dss(
    id="linecode-16sq",
    nphases=1,
    r1=rlf.Q_(1.150, "ohm/km"),
    x1=rlf.Q_(0.083, "ohm/km"),
    r0=rlf.Q_(1.200, "ohm/km"),
    x0=rlf.Q_(0.083, "ohm/km"),
    c1=rlf.Q_(3.400, "nF/km"),
    c0=rlf.Q_(1.600, "nF/km"),
    basefreq=rlf.Q_(50, "Hz"),
    normamps=rlf.Q_(400, "A"),
    linetype="OH",
)

# Lines
line_ab = rlf.Line(
    "lineA_B", bus1=bus_a, bus2=bus_b, phases="abc", parameters=lp_240, length=rlf.Q_(1, "km"), ground=ground
)
line_bc = rlf.Line(
    "lineB_C", bus1=bus_b, bus2=bus_c, phases="a", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)
line_bd = rlf.Line(
    "lineB_D", bus1=bus_b, bus2=bus_d, phases="b", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)
line_be = rlf.Line(
    "lineB_E", bus1=bus_b, bus2=bus_e, phases="c", parameters=lp_16, length=rlf.Q_(10, "m"), ground=ground
)

In the final modelling step, we will create the loads. Since all the loads on the network are constant power loads, 
they can be modelled using the [`PowerLoad`](https://roseau-load-flow.roseautechnologies.com/models/Load/PowerLoad.html)
 class of _RLF_. However, because the `powers` parameter in the 
 [`PowerLoad`](https://roseau-load-flow.roseautechnologies.com/models/Load/PowerLoad.html) class takes in apparent 
 power, we write a small function `complex_power` to convert the peak demand (kW) given in the loads table to apparent 
 power at the corresponding power factor.


In [ ]:
# Convert Active Power to Apparent Power
def complex_power(p: float, pf: float) -> complex:
    phi = np.arccos(pf)
    q = p * np.tan(phi)
    return p + 1j * q


# Loads
load1 = rlf.PowerLoad(id="load1", bus=bus_c, phases="an", powers=[complex_power(7e3, 0.95)])
load2 = rlf.PowerLoad(id="load2", bus=bus_d, phases="bn", powers=[complex_power(6e3, 0.95)])
load3 = rlf.PowerLoad(id="load3", bus=bus_e, phases="cn", powers=[complex_power(8e3, 0.95)])

### Running a Load Flow Simulation

As shown in tutorial 1, before running a load flow simulation in _RLF_, we must first create the network 
(`ElectricalNetwork`) from the disjointed network elements that have been instantiated in the preceding steps. 
The straightforward way to do this is to use the `from_element` method of the `ElectricalNetwork` class. This method 
allows us to create the entire network from a single bus and adds all the other elements procedurally.


In [ ]:
en = rlf.ElectricalNetwork.from_element(initial_bus=source_bus)
en

Then, we can run the load flow by simply calling the `solve_load_flow` method of the network.


In [ ]:
en.solve_load_flow()

### Accessing and Comparing Results

Here, we compare the results from the _OpenDSS_ simulations with that of _RLF_. In each of the subsequent steps, note 
the ease of accessing results in _RLF_ compared to constantly setting active elements in _OpenDSS_.


First, we compare the active and reactive powers calculated by the _RLF_ solver. To access the active power of the 
load, we simply need to use the `res_powers` property of each load. This returns an _RLF_ Quantity object which 
contains the incoming/outgoing apparent power (in VA) of the load. These powers can simply be converted into kVA 
using the `m_as` method of the `Quantity` class. The sum of these powers gives the total power dissipated by the load. 
It can be observed that the active and reactive powers are exactly the same as that of _OpenDSS_.


In [ ]:
# Active and Reactive Powers of the Loads
table_rlf_results: list[str | None] = []
for load in (load1, load2, load3):
    table_rlf_results.append(None)

    print(f"{load.id}:  ")
    load_powers = load.res_powers.m_as("kVA").sum()
    load_value = f"{load_powers.real:.3f} kW"
    print(f"{active_power_demand_title} = {load_value}")
    table_rlf_results.append(load_value)

    load_value = f"{load_powers.imag:.3f} kVAr"
    print(f"{reactive_power_demand_title} = {load_value}")
    table_rlf_results.append(load_value)

Next, we examine the bus voltages of the load buses which we can access by using the `res_voltages` property of the loads. Then, we simply convert these values into per-unit values using a base value of ($\frac{400}{\sqrt{3}}$) to compare to the OpenDSS results which are given in per-unit by default. Again, we see that the results are the same with that of OpenDSS


In [ ]:
for load in (load1, load2, load3):
    table_rlf_results.append(None)

    voltages_pu = abs(load.res_voltages.m[0]) / (un_lv.m_as("V") / np.sqrt(3))
    voltage_title = f"The voltage of the bus connected to {load.id}"
    voltage_value = f"{voltages_pu:.3f} pu"
    print(f"{voltage_title} = {voltage_value}")
    table_rlf_results.append(voltage_value)

Similarly, we can obtain the bus voltage at bus A using the `res_voltages` property of the bus and converting the values into per-unit values in the same manner. It can be seen that the results are once again the same.


In [ ]:
print("Voltage magnitudes at bus A:  ")
table_rlf_results.append(None)
bus_a_voltages_pu = abs(bus_a.res_voltages.m) / (un_lv.m_as("V") / np.sqrt(3))
for i in range(3):
    voltage_value = f"{bus_a_voltages_pu[i]:.3f} pu"
    table_rlf_results.append(voltage_value)
    print(f"Voltage magnitude - phase {i+1} = {voltage_value}")

Then we extract the power supplied to the transformer as well as the power losses using the `res_powers` and 
`res_power_losses` properties respectively. The active and reactive power in each phase as well as the power lost are
 derived as shown below. Here we see a bit of difference with the _OpenDSS_ results in some parameters. This is due to 
 imperfections in converting an _OpenDSS_ transformer specification into an _RLF_ one. However, these differences in 
 results are very small, on the order of $10^{-3}$ and are not significant.


In [ ]:
transformer_powers = transformer.res_powers[1].m_as("kVA")
transformer_power_losses = transformer.res_power_losses.m_as("kVA")
print("Results of the transformer LVTR: ")
table_rlf_results.append(None)
for i in range(3):
    transformer_value = f"{abs(transformer_powers[i].real):.5f} kW"
    table_rlf_results.append(transformer_value)
    print(f"Active power (P) supplied to phase {i+1} = {transformer_value}")
for i in range(3):
    transformer_value = f"{abs(transformer_powers[i].real):.5f} kVAr"
    table_rlf_results.append(transformer_value)
    print(f"Reactive power (Q) supplied to phase {i+1} = {transformer_value}")
print()

Finally, we investigate the line losses calculated during the _RLF_ simulation. We can access the line losses using 
the `res_lines` property of the network itself. This returns a dataframe where we can extract the series losses of 
the lines. When compared to _OpenDSS_ results, it can be seen that both are the exact same values.  


In [ ]:
total_line_loss = en.res_lines["series_losses"].sum() / 1e3  # Convert to kVA
print("Results of the Power Losses: ")
table_rlf_results.append(None)
line_value = f"{abs(total_line_loss.real):.3f} kW"
table_rlf_results.append(line_value)
print(f"Active power (P) losses = {line_value}")
line_value = f"{abs(total_line_loss.imag):.3f} kVAr"
table_rlf_results.append(line_value)
print(f"Reactive power (Q) losses = {line_value}")

## Comparison

Please find below a table which summarizes the results of the load flow solvers.


In [ ]:
table = Table(title="Comparison of results")

table.add_column("Title", justify="left", no_wrap=True)
table.add_column(Text("OpenDSS", style="italic"), justify="right", style="green")
table.add_column(Text("Roseau Load Flow", style="italic"), justify="right", style="green")


for title, opendss_value, rlf_value in zip(table_titles, table_opendss_results, table_rlf_results, strict=True):
    if opendss_value is None:
        table.add_section()
        table.add_row(Text(title, style="bold"))
    elif rlf_value == opendss_value:
        table.add_row(title, opendss_value, rlf_value)
    else:
        table.add_row(title, Text(opendss_value, style="red"), Text(rlf_value, style="red"))

table

## Conclusion

This tutorial has demonstrated the modelling flexibility and interoperability of the _Roseau Load Flow_ solver. We've 
been able to model an LV network with an earth return system while using parameters specified in _OpenDSS_ format. The
 results calculated by _RLF_ have also been shown to be the same as that of OpenDSS showing the effectiveness of our 
 solver.
